<a href="https://colab.research.google.com/github/ldeluigi/supermarket-2077-product-vision/blob/master/ProductDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Product Detection

## Download datasets

In [ ]:
!rm -rf sample_data
!gdown --id 1fDr4g4wbnSRkuCYyS3wpuJS7Ax22bVB_ -O all.zip
!unzip -oq all.zip

%matplotlib inline

## Imports

In [ ]:
import scipy.io
import os
import re
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

## Read store data

In [ ]:
def read_store_data(storename):
  dirname = storename
  dirname_anno = os.path.join(dirname, 'annotation')
  dirname_images = os.path.join(dirname, 'images')
  directory_anno = os.fsencode(dirname_anno)
  directory_images = os.fsencode(dirname_images)

  result = []

  for file in os.listdir(directory_anno):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"): 
      mat = scipy.io.loadmat(os.path.join(dirname_anno, filename))
      number = re.search(r'^anno.(\d+).mat$', filename).group(1)
      img = cv2.imread(os.path.join(dirname_images, number + '.jpg'))

      img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img_annotation = mat['annotation'][0, 0]
      bbox = list(map(lambda x: x[0], img_annotation[0][0]))
      labels = list(map(lambda x: str(x[0][0][0]), img_annotation[1][0]))
      class_indexes = list(img_annotation[2][0])
      
      result.append((img_rgb, bbox, class_indexes, labels))
  return result, ['image', 'bboxes', 'class_indexes', 'labels']

## Read training data

In [ ]:
dirname = 'Training'

def read_classes():
  mat = scipy.io.loadmat(os.path.join(dirname, 'TrainingClassesIndex.mat'))
  classes = list(map(lambda x: x[0], mat['classes'][0]))
  return dict(enumerate(classes, start=1))

def read_training_data(classes):
  result = []
  for class_index, class_name in classes.items():
    dirname_images = os.path.join(dirname, class_name)
    directory_images = os.fsencode(dirname_images)
    for file in os.listdir(directory_images):
      img = cv2.imread(os.path.join(dirname_images, os.fsdecode(file)))
      img_rgb =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      result.append((img_rgb, class_index))
  return result, ['image', 'class_index']

## Prepare products dataset

In [ ]:
def create_dataset(data, columns):
  return pd.DataFrame(data, columns=columns)

classes = read_classes()
products = create_dataset(*read_training_data(classes))
store = create_dataset(*read_store_data('store1'))

def class_name(class_index):
  return classes[class_index] if class_index >= 0 else None

products['class'] = products.apply(lambda x: class_name(x['class_index']), axis=1)

In [ ]:
def summary_of_dataset(dataset):
  print('Information on classes:')
  info = dataset['class'].value_counts()
  print(info)
  print(info.describe())

summary_of_dataset(products)

## Data visualization

In [ ]:
def plot_grid(indexes, columns, dataset, draw_item):
  fig = plt.figure(figsize=(12, 6), dpi=120)
  # fig.subplots_adjust(hspace=0.2)
  for index, i_img in enumerate(indexes, start=1):
    sp = fig.add_subplot(math.ceil(len(indexes) / columns), columns, index)
    row = dataset.iloc[i_img]
    draw_item(row, sp)

In [ ]:
def show_products_with_class(indexes, columns, dataset):
  def show_single_product_with_class(row, sp):
    plt.axis('off')
    plt.imshow(row['image'])
    sp.set_title(row['class'], fontsize=10)
  plot_grid(indexes, columns, dataset, show_single_product_with_class)

show_products_with_class(np.random.randint(0, len(products), 6), 2, products)

In [ ]:
color = (255, 255, 0)

def show_shelf_with_bbox_and_classes(indexes, columns, dataset):
  def show_single_shelf(row, sp):
    img = row['image'].copy()
    for i, bbox in enumerate(row['bboxes']):
      [xmin, xmax, ymin, ymax] = bbox
      height, width, _ = img.shape
      xmin = int(xmin * width)
      xmax = int(xmax * width)
      ymin = int(ymin * height)
      ymax = int(ymax * height)
      thickness = round(max(width, height) * 0.01)
      cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, thickness)
      cv2.putText(img,
                  class_name(row['class_indexes'][i]),
                  (round(xmin + thickness * 1.5), round(ymin + thickness * 3)),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  2.5,
                  color,
                  thickness=round(thickness/2))
    plt.axis('off')
    plt.imshow(img)
  plot_grid(indexes, columns, dataset, show_single_shelf)

show_shelf_with_bbox_and_classes(np.random.randint(0, len(store), 1), 2, store)